In [45]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time


In [2]:
# read from sgpr1.html
with open("sgpr1.html", "r") as f:
    soup = BeautifulSoup(f.read(), 'html.parser')

In [8]:
soup.find_all("tr")[0]


<tr>
<th width="50">修改</th>
<th width="80">用户名</th>
<th width="320">条件</th>
<th width="60">结果</th>
<th width="100">申请日期</th>
<th width="100">结束日期</th>
<th width="140">更新时间</th>
</tr>

In [6]:
for tr in soup.find_all('tr'):
    row_data = {}
    a_tag = tr.find('a', {'class': 'edit'})
    if a_tag:
        row_data['id'] = a_tag.get('data-id')  # Extract data-id
    
    td_tags = tr.find_all('td')
    for i, td in enumerate(td_tags):
        row_data[f'td_{i+1}'] = td.text.strip()  # Extract text from td tags
    print(row_data)

{}
{'id': '1001434', 'td_1': '\ue254', 'td_2': '我要疯了', 'td_3': '应该是2023-10-28到2023-11-01之间某天出现4条线 我11月1号看到 不可以重新申请。。。。老天保佑！！！ 都快没麻木了。。。。', 'td_4': '等待', 'td_5': '2022-05-10', 'td_6': '', 'td_7': '2023-11-02 15:29'}
{'id': '1001433', 'td_1': '\ue254', 'td_2': '求过', 'td_3': 'PHD, EP Pass, IT, 在新工作4年+，妈妈+2个孩子, 13K。', 'td_4': '等待', 'td_5': '2023-05-30', 'td_6': '', 'td_7': '2023-11-01 22:55'}
{'id': '1001432', 'td_1': '\ue254', 'td_2': 'I know will pass der~', 'td_3': 'Malaysian Chinese F, 24 yo, S-pass, MY Diploma+LCCI+SPM double cert, designer, 2.5k basic only, A lot of charity records in SG, not sure useful or not', 'td_4': '等待', 'td_5': '2023-02-16', 'td_6': '', 'td_7': '2023-11-01 15:29'}
{'id': '1001352', 'td_1': '\ue254', 'td_2': '。。。', 'td_3': '一家三口 中国人 建筑业 NUS Part time Degree 来新10年 主申请人 收入6k+ 第四次申请 希望这次过过过', 'td_4': '等待', 'td_5': '2023-06-14', 'td_6': '', 'td_7': '2023-11-01 14:02'}
{'id': '1001430', 'td_1': '\ue254', 'td_2': '无语人士', 'td_3': '马来西亚人 ， S Pass ， 普通LCCI 文凭', 'td_4': 

In [36]:
# Connect to SQLite database (or create it)
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Get column headers and create table
headers = ["id"] + [header.text for header in soup.find_all('th')]
print(headers)
table_create_query = f"""CREATE TABLE IF NOT EXISTS data_table (
                        id TEXT PRIMARY KEY,
                        {', '.join([f'"{header}" TEXT' for header in headers[1:]])}
                        );"""
cursor.execute(table_create_query)
conn.commit()

['id', '修改', '用户名', '条件', '结果', '申请日期', '结束日期', '更新时间']


In [38]:
# writes data to db
for tr in soup.find_all('tr')[0:3]:
    a_tag = tr.find('a', {'class': 'edit'})
    if a_tag:
        id_ = a_tag.get('data-id')  # Extract data-id

    td_tags = tr.find_all('td')
    if td_tags:
        row_data = [id_] + [td.text.strip() for td in td_tags]
        cursor.execute(f"INSERT OR REPLACE INTO data_table VALUES({', '.join(['?' for _ in row_data])});", row_data)
conn.commit()


In [39]:
cursor.execute(f"SELECT `id`, `条件`, `结果`, `申请日期`, `结束日期` FROM data_table").fetchall()

[('1001434',
  '应该是2023-10-28到2023-11-01之间某天出现4条线 我11月1号看到 不可以重新申请。。。。老天保佑！！！ 都快没麻木了。。。。',
  '等待',
  '2022-05-10',
  ''),
 ('1001433',
  'PHD, EP Pass, IT, 在新工作4年+，妈妈+2个孩子, 13K。',
  '等待',
  '2023-05-30',
  '')]

In [43]:
def save_data_to_db(conn, cursor, soup: BeautifulSoup):
    # writes data to db
    id_results = []
    for tr in soup.find_all('tr'):
        a_tag = tr.find('a', {'class': 'edit'})
        id_= None
        if a_tag:
            id_ = a_tag.get('data-id')  # Extract data-id

        td_tags = tr.find_all('td')
        if id_ and td_tags:
            id_results.append(id_)
            row_data = [id_] + [td.text.strip() for td in td_tags]
            cursor.execute(f"INSERT OR REPLACE INTO data_table VALUES({', '.join(['?' for _ in row_data])});", row_data)
    conn.commit()
    return id_results




In [49]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import time

base_url = 'http://sgprapp.com/listPage?'  # Base url

# Connect to SQLite database (or create it)
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Get column headers and create table
headers = ["id"] + [header.text for header in soup.find_all('th')]
print(headers)
table_create_query = f"""CREATE TABLE IF NOT EXISTS data_table (
                        id TEXT PRIMARY KEY,
                        {', '.join([f'"{header}" TEXT' for header in headers[1:]])}
                        );"""
cursor.execute(table_create_query)
conn.commit()

# Start from page 0 and increment until empty page is found
page_number = 0
while True:
    print(f"Scraping page {page_number}")
    response = requests.get(base_url + f'page={page_number}')
    soup = BeautifulSoup(response.text, 'html.parser')
    tr_tags = soup.find_all('tr')


    ids = save_data_to_db(conn, cursor, soup)
    print(f"Saved {len(ids)} records, ids: {ids}")
    # Break the loop if page does not contain any data
    if len(ids) == 0:
        print("Exited")
        break

    # Sleep 2
    time.sleep(2)
    # Increment page number for the next iteration
    page_number += 1

['id', '修改', '用户名', '条件', '结果', '申请日期', '结束日期', '更新时间']
Scraping page 0
Saved 500 records, ids: ['1001434', '1001433', '1001432', '1001352', '1001430', '1001146', '1001426', '1001423', '1001422', '1001421', '1001420', '1001419', '1001418', '1001171', '1001415', '1001293', '1001412', '1001411', '1001231', '1001408', '1001410', '1001409', '1001316', '1001358', '1001404', '1001406', '1001400', '1001405', '1001403', '1001402', '1001399', '1001401', '1001347', '1001398', '1001395', '1001397', '1001396', '1001232', '1001357', '1001393', '1001392', '1001391', '1001390', '1001181', '1001321', '1001388', '1001265', '1001386', '1000970', '1001150', '1001384', '1001383', '1001372', '1001382', '1001380', '1001377', '1001376', '1001375', '1001374', '1001373', '1001223', '1001368', '1001367', '1001366', '1001365', '1001363', '1001343', '1001351', '1000979', '1001211', '1001354', '1001145', '1001350', '1001346', '1001348', '1001161', '1001345', '1001344', '1001342', '1001341', '1001340', '1001322', '

In [51]:
cursor.execute(f"SELECT count(1) FROM data_table").fetchall()

[(2270,)]